In [ ]:
from pybeam.datamodels import nodes_from_csv, elements_from_csv
from pybeam.assembly import assemble_system_matrices, reindex_dof
from pybeam.simulation import simulate
from pybeam.modal_parameters import get_modal_parameters
from pybeam._utilities import pprint_array
from pybeam.plotting import plot_structure, plot_modeshape, set_style, plot_deformations
import matplotlib.pyplot as plt

%matplotlib qt5
set_style()
import numpy as np

import logging

logging.getLogger("pybeam").setLevel(logging.INFO)

### System assembly and computation of mode shapes

In [ ]:
from pybeam.datamodels import nodes_from_csv, elements_from_csv
from pybeam.assembly import assemble_system_matrices, reindex_dof
from pybeam.modal_parameters import get_modal_parameters

nodes = nodes_from_csv("data/clamped_beam/nodes.csv")
elements = elements_from_csv("data/clamped_beam/elements.csv", nodes)

stiffness, mass = assemble_system_matrices(elements)
damping = 0.01 * mass + 0.001 * stiffness

elements = reindex_dof(elements)
eigenfrequencies, modeshapes = get_modal_parameters(stiffness, mass)

### Plot mode shapes

In [ ]:
import matplotlib.pyplot as plt
from pybeam.plotting import plot_structure, plot_modeshape

plt.figure()
plot_structure(elements, node_labels=False, element_labels=False)
plot_modeshape(elements, modeshapes, mode=0, scale=1)

### Time response simulation

In [ ]:
# Number of DOF
n_dof = stiffness.shape[0]

# Time step size
dt = 0.005

# End time
t_end = 2
# Time steps
time = np.arange(start=0, stop=t_end + dt, step=dt)

# Loading
loads = np.zeros((n_dof, len(time)))
# loads[9, :] = 100 * np.cos((eigenfrequencies[0] + eigenfrequencies[1]) / 2 * time)

# loads[13, :] = 50 * np.cos(0.8 * eigenfrequencies[2] * time)

# Initial conditions
x_0 = np.zeros((n_dof,))
x_0[7] = 0.5
v_0 = np.zeros((n_dof,))

# Integration parameter
beta = 1 / 4

x, v, a = simulate(
    stiffness=stiffness,
    mass=mass,
    damping=damping,
    initial_disp=x_0,
    initial_vel=v_0,
    loads=loads,
    time=time,
    beta=beta,
    gamma=1 / 2,  # Don't change this unless you know what you're doing
)

### Virtual sensing

In [ ]:
import numpy as np

n_time_steps = x.shape[1]  # The number of time steps
n_dof = modeshapes.shape[0]  # The number of model-based DOF

measured_dof = (1, 2, 4, 5, 7)  # DOF where we have sensors
estimated_dof = (6, 7, 8)  # DOF where we wish to estimate the response

# Helper variable
sensor_map = {dof: i for i, dof in enumerate(estimated_dof)}

phi_m = modeshapes[measured_dof, :]
x_m = x[measured_dof, :]

phi_v = modeshapes[estimated_dof, :]
x_v = np.zeros((len(estimated_dof), n_time_steps))

q = np.zeros((n_dof, n_time_steps), dtype=np.float64)

# Loop over time steps, determine q and x_v at each time step
for i, x_im in enumerate(x_m.T):
    q[:, i] = np.linalg.pinv(phi_m).dot(x_im)
    x_v[:, i] = phi_v.dot(q[:, i])

### Inspect results

In [ ]:
import matplotlib.pyplot as plt

dof_to_plot = 8


plt.figure()
ax = plt.gca()

# Adjust the look of the plot
ax.set_xlabel("Time")
ax.set_ylabel("Displacement")
ax.set_xlim(0, time[-1])

ax.plot(time, x[dof_to_plot, :], color="white")
ax.plot(time, x_v[sensor_map[dof_to_plot], :], color="blue", linestyle="dashed")
plt.show()